In [1]:
import math
import json
import scipy
import pickle
import numpy as np 
import pandas as pd

from operator import itemgetter
from tinydb import TinyDB, Query

In [2]:
from functions import *

In [ ]:
### Print recomended titles for tfg docs NO BORRAR

datasetId = 5

db = TinyDB("../data/catalog.json")

with open("../data/pairwise_similarity_titles.pkl", "rb") as input_file:
    pairwise_similarity_title = pickle.load(input_file)
    pairwise_similarity_title = np.array(pairwise_similarity_title)
    
similarity_vector = list(pairwise_similarity_title[datasetId])
similarity_vector_sorted = sorted(similarity_vector, reverse=True)
sorted_ind = [similarity_vector.index(i) for i in similarity_vector_sorted]

itemObj = db.search(Query().index == int(datasetId))[0]
print(itemObj["title"])
print("----------------------")
print("----------------------")

for value in sorted_ind[0:6]:
    itemObj = db.search(Query().index == int(value))[0]
    print(itemObj["title"])
    print(similarity_vector[value])
    print("----------------------")

In [ ]:
### Print recomended descriptions for tfg docs NO BORRAR

datasetId = 5

db = TinyDB("../data/catalog.json")

with open("../data/pairwise_similarity_descriptions.pkl", "rb") as input_file:
    pairwise_similarity_desc = pickle.load(input_file)
    pairwise_similarity_desc = np.array(pairwise_similarity_desc)
    
similarity_vector = list(pairwise_similarity_desc[datasetId])
similarity_vector_sorted = sorted(similarity_vector, reverse=True)
sorted_ind = [similarity_vector.index(i) for i in similarity_vector_sorted]

itemObj = db.search(Query().index == int(datasetId))[0]
print(itemObj["description"])
print("----------------------")
print("----------------------")

for value in sorted_ind[0:6]:
    itemObj = db.search(Query().index == int(value))[0]
    print(itemObj["description"])
    print(similarity_vector[value])
    print("----------------------")

In [ ]:
### Print recomended descriptions for tfg docs NO BORRAR

COEF = 0.7

itemId = 5

# Retrieve similaity matrices
with open("../data/pairwise_similarity_titles.pkl", "rb") as input_file:
    pairwise_similarity_title = pickle.load(input_file)
    pairwise_similarity_title = np.array(pairwise_similarity_title)
with open("../data/pairwise_similarity_descriptions.pkl", "rb") as input_file:
    pairwise_similarity_desc = pickle.load(input_file)
    pairwise_similarity_desc = np.array(pairwise_similarity_desc)

# Calculate similarity
pairwise_similarity = np.add(COEF * pairwise_similarity_title, (1 - COEF) * pairwise_similarity_desc)
pairwise_similarity = pairwise_similarity / 2

# Change format and sort
similarity_vector = pairwise_similarity[itemId] #.toarray()
similarity_vector = list(similarity_vector)
similarity_vector_sorted = sorted(similarity_vector, reverse=True)
sorted_ind = [similarity_vector.index(i) for i in similarity_vector_sorted]


itemObj = db.search(Query().index == int(datasetId))[0]
print(itemObj["title"])
print(itemObj["description"])
print("----------------------")
print("----------------------")

for value in sorted_ind[0:6]:
    itemObj = db.search(Query().index == int(value))[0]
    print(itemObj["title"])
    print(itemObj["description"])
    print(similarity_vector[value])
    print("----------------------")

------------------------------------------------------------------------------------------

In [11]:
itemId = 5

db = TinyDB("../data/catalog.json")
itemObj = db.search(Query().index == int(itemId))[0]
itemObj['columns'] = getcolumnNames(itemId)
itemObj['profileRecomendations'] = profileRecomendations(itemId, itemObj['columns'])

In [16]:
itemObj['columns']

[{'type': ' categorical',
  'name': 'Numero_expedient',
  'similar': [{'datasetId': '2',
    'datasetTitle': 'Descripció de la causalitat dels accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona',
    'datasetUrl': 'https://opendata-ajuntament.barcelona.cat/data/ca/dataset?q=&name=accidents-causes-gu-bcn',
    'name': 'Numero_expedient',
    'value': 1.0},
   {'datasetId': '3',
    'datasetTitle': 'Accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona',
    'datasetUrl': 'https://opendata-ajuntament.barcelona.cat/data/ca/dataset?q=&name=accidents-gu-bcn',
    'name': 'Numero_expedient',
    'value': 1.0},
   {'datasetId': '5',
    'datasetTitle': 'Accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona segons tipologia',
    'datasetUrl': 'https://opendata-ajuntament.barcelona.cat/data/ca/dataset?q=&name=accidents-tipus-gu-bcn',
    'name': 'Numero_expedient',
    'value': 1.0},
   {'datasetId': '6',
    'datasetTitle': 'Vehicles implicats en a

In [15]:
itemId = 5
columns = itemObj['columns']

SIZE = 4                  # number of datasets to return
similarities = dict()     # keep track of column dictionaries for each dataset

for column in columns:
    for similarColumn in column["similar"]:
        # Only keep values larger that 0.9/100 
        # to avoid considering columns with low similarity
        if similarColumn['value'] > 0.9:

            columnObj = {
                "originalColumn": column["name"],
                "similarColumn": similarColumn['name'],
                "value": similarColumn['value']
            }

            if similarColumn['datasetId'] in similarities: 
                similarities[similarColumn['datasetId']].append(columnObj)
            else: 
                similarities[similarColumn['datasetId']] = [columnObj]
                
result = []                
for key in list(similarities.keys()):

    datasetSimilarity = 0
    commonColumns = 0 # number of high similarity columns
    similarColumnslist = []
    for columnValue in similarities[key]:  
        datasetSimilarity += float(columnValue["value"])
        commonColumns += 1
        similarColumnslist.append([{
            "originalColumn": columnValue["originalColumn"], 
            "similarColumn": columnValue["similarColumn"]
        }])

    # Get information about dataset for the frontend
    db = TinyDB("../data/catalog.json")
    datasetObj = db.search(Query().index == int(key))[0]

    result.append({
        "datasetId": key,
        "datasetInformation": {
            "title": datasetObj["title"],
            "description": datasetObj["description"],
            "origin": datasetObj["origin"]
        },
        "datasetSimilarity": datasetSimilarity,
        "jaccard": commonColumns 
        "countSimilarColumns": len( [key]),
        "similarColumns": similarColumnslist
    })

In [10]:
result

[{'datasetId': '2',
  'datasetInformation': {'title': 'Descripció de la causalitat dels accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona',
   'description': 'Llistat de de la causalitat dels accidents gestionats per la Guàrdia Urbana a la ciutat de Barcelona. Un accident pot tenir una o més causes mediates\xa0les quals fan referència a factors\xa0externs del resultat en\xa0temps, lloc o grau (ex: Alcoholèmia i Excés de velocitat o inadequada).\xa0     ',
   'origin': 'Barcelona'},
  'datasetSimilarity': 16.0,
  'countSimilarColumns': 1,
  'similarColumns': [[{'originalColumn': 'Numero_expedient',
     'similarColumn': 'Numero_expedient'}],
   [{'originalColumn': 'Nom_districte', 'similarColumn': 'Nom_districte'}],
   [{'originalColumn': 'Nom_barri', 'similarColumn': 'Nom_barri'}],
   [{'originalColumn': 'Nom_carrer', 'similarColumn': 'Nom_carrer'}],
   [{'originalColumn': 'Num_postal_caption', 'similarColumn': 'Num_postal '}],
   [{'originalColumn': 'Nom_mes', 'simil